In [1]:
import numpy as np
import pandas as pd

In [2]:
# represent cards as bits


In [3]:
_num_ranks = 13
_num_suits = 4
_suit_base = "A"

In [4]:
def suit_to_str(s):
    return chr(ord(_suit_base) + s)

### intro
- A hand is composed of multiple cards (at least 1)
- We keep count of the number of each suit, number of each rank
- We track straights using bits (each suit's cards is represented by a 13 bit #)j

- hand = (52 bits)

- straight flushes
    - check each of the bit patterns against 0b11111 (4 * 10 = 40)
- 4 of a kind
    - check each of the bit patterns (0x1...1...1...1): 13
- full house

- flush
    - check bit_count of each suit (4 ops)
        - for each one where it is at least 5, take the max (this is the "biggest one") (4 ops for max)
        - truncate things that are below the 5 largest bits (? ops)
- straight
    - apply 'or' on all ranks and check each bit pattern (4 + 10 = 14 ops)
- trips
    - identify largest trips

In [5]:
def get_highest_bit(bits):
    return bits.bit_length() - 1

def get_num_set_bits(bits):
    return bits.bit_count()

def get_nth_bit(bits, n):
    assert (1 <= n <= bits.bit_length())
    for i in range(n-1):
        bits -= (1 << (bits.bit_length()-1))
    return bits.bit_length() - 1

In [57]:
_straight_base = (1 << 5) - 1
straight_masks = [_straight_base << i for i in range(_num_ranks + 1 - 5)]
straight_masks = [0b1000000001111] + straight_masks # A2345

score_to_str = ["highcard", "pair", "twopair", "threes", "straight", "flush", "fullhouse", "fours", "straightflush"]
str_to_score = {s : i for i, s, in enumerate(score_to_str)}


In [19]:
[bin(b) for b in straight_masks]

['0b1000000001111',
 '0b11111',
 '0b111110',
 '0b1111100',
 '0b11111000',
 '0b111110000',
 '0b1111100000',
 '0b11111000000',
 '0b111110000000',
 '0b1111100000000']

In [60]:
def is_straight_flush(bits):
    best_res = 0
    for i in range(_num_suits):
        print(bin(bits))
        # note that straight_masks are in order from worst to best
        for msk in straight_masks:
            # check if mask works
            res = (bits & msk)
            success = res == msk
            # print(i, bin(bits & (1 << 13 - 1)), bin(msk), bin(res))
            if success:
                print(i, bin(msk))
                best_res = max(res, best_res)
        bits = bits >> _num_ranks
        
    if best_res != 0:
        best_res += str_to_score["straightflush"] << _num_ranks
        
    return best_res


In [62]:
res = is_straight_flush(0b1000000001111 + (0b0001111100000 << 13) + (0b111110000000 << 26) + ((0b11111 << 8) << 39))
bin(res)

0b1111100000000011111000000000011111000001000000001111
0 0b1000000001111
0b111110000000001111100000000001111100000
1 0b1111100000
0b11111000000000111110000000
2 0b111110000000
0b1111100000000
3 0b1111100000000


'0b10001111100000000'